API 키 로드
`.env` 에 설정되어 있어야 합니다.
```
OPENAI_API_KEY=sk-proj-******** # Your Key
```

In [17]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [18]:
import oracledb
import os
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [19]:
# 1. 데이터베이스 연결

# SQL 파일 읽기
sql_path = '../.db/oracle/product.sql'
with open(sql_path, 'r', encoding='utf-8') as f:
    sql = f.read().strip()

# Oracle 데이터베이스 연결 정보
connection = oracledb.connect(
    user=os.getenv("ORACLE_USER"),
    password=os.getenv("ORACLE_PASSWORD"),
    host=os.getenv("ORACLE_HOST"),
    port=os.getenv("ORACLE_PORT"),
    sid=os.getenv("ORACLE_SID")
    )

cursor = connection.cursor()
cursor.execute(sql)
rows = cursor.fetchall()

# 연결 종료
cursor.close()
connection.close()

# documents 리스트 생성
documents = []
metadatas = []
ids = []

"""
- 포맷: {BRND_NM}의 {ARTC_INFO}, {GOODS_NM}.
- 예시: 삼성전자의 TV, QLED 8K 스마트TV.
"""
for row in rows:
    GOODS_NO, GOODS_NM, BRND_NM, ARTC_INFO = row
    metadatas.append({"GOODS_NO": GOODS_NO})
    documents.append(f"{BRND_NM}의 {ARTC_INFO}, {GOODS_NM}")
    ids.append(str(GOODS_NO))

documents

['신일의 겨울제품(히터), 선풍기형 세라믹 전기히터 SEH-900BKK [3단계 온도조절 / 안전장치 / 3시간타이머]',
 '하이메이드의 전자레인지(전자식), ***BEST*** 1구 전기레인지 모음 [인덕션 / 하이라이트]',
 '칸쿄의 건강기기(공기청정기), 터뷸런스 공기청정기 TB502',
 '퓨어시스의 건강기기(공기청정기), 공기정화기 ALL-IN-ONE-30',
 '퓨어시스의 건강기기(공기청정기), 스탠드용 공기정화기 ALL-IN-ONE-50',
 '퓨어시스의 건강기기(공기청정기), 벽걸이(스탠드) 겸용 공기정화살균기 all-in-one 20',
 '이지랩의 건강기기(헬스케어), 탄소 온열 벨트& 탄소 매트/패드 6종',
 'POUT의 디지털기타(기타), ★★미니 공기청정기 NOSE1★★',
 '삼성전자의 TV(QLEDTV), 189cm QLED TV QN75Q6FNAFXKR',
 '삼성전자의 TV(QLEDTV), 163cm QLED TV QN65Q9FNAFXKR',
 'LG전자의 TV(UHD TV), *최대혜택가 1,528,000원/빠른배송가능* 189cm UHD TV 75UK7400KNA [초대화면/매직리모컨/HDR 지원/스마트 사운드]',
 '삼성전자의 TV(QLEDTV), 189cm QLED TV QN75Q9FNAFXKR',
 '아도니스의 TV(UHD TV), 165cm UHD TV / TS-650CUHD',
 '아도니스의 TV(UHD TV), 165cm UHD TV / TS-650UHD',
 '위니아대우의 TV(UHD TV), 163cm UHD TV U65U8210KK',
 'LG전자의 TV(UHD TV), 163cm SUHD TV 65SK9600KNA',
 '위니아딤채의 TV(UHD TV), 165cm UHD TV / WV65BU4000 (전용 액세서리 구매가능)',
 '위니아딤채의 TV(UHD TV), 月29,417원 (36개월 무이자 적용시) 165cm UHD TV / WV65BU4000 스탠드 전문기사 설치',
 '위니아딤채의 TV

In [20]:
# 2. OpenAI 임베딩 객체 생성 (model 지정)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

In [21]:
#  3. 벡터스토어를 생성합니다.

# DB 경로 설정
persist_directory = "../.db/faiss"

vectorstore = FAISS.from_texts(
    texts=documents,
    embedding=embeddings,
    metadatas=metadatas,
    ids=ids
)

# 로컬에 저장
vectorstore.save_local(persist_directory)

print(f"총 {len(documents)}개의 상품 정보가 FAISS에 저장되었습니다.")

총 451개의 상품 정보가 FAISS에 저장되었습니다.


In [22]:

# 4. Vectorstore 검색
vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "오늘 날씨가 덥네"  # 예시 쿼리: 원하는 검색어로 변경하세요

# 쿼리와 유사한 상품 5개 검색
results_with_score = vectorstore.similarity_search_with_score(
    query=query, k=5  # 반환할 결과 수
)

# 결과 출력
print(f"'{query}'와 유사한 상품 검색 결과 (최대 5개):")
print("-" * 60)

if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"[결과 {i}]")
        print(f"상품정보: {doc.page_content}")
        print(f"상품번호: {doc.metadata['GOODS_NO']}")
        print(f"유사도 점수: {score}")  # 낮을수록 더 유사함
        print("-" * 60)
else:
    print("검색 결과가 없습니다.")


'오늘 날씨가 덥네'와 유사한 상품 검색 결과 (최대 5개):
------------------------------------------------------------
[결과 1]
상품정보: 이노크아든의 겨울제품(미사용6), 4L 클린볼 초음파 간편세척 가습기 IA-H100 (화이트, 블랙)  
상품번호: 0001050042
유사도 점수: 1.4798052310943604
------------------------------------------------------------
[결과 2]
상품정보: O&A의 겨울제품(미사용6), 안개가습기 OA-HM034(1000ML) OA-HM024(500ML) [ 2가지 가습모드／ 최대 9시간 타이머/ 무드등]
상품번호: 0005526547
유사도 점수: 1.4859747886657715
------------------------------------------------------------
[결과 3]
상품정보: 나눔의 None, [필터] 나눔 러브팟 자연 가습기 4종
상품번호: 0000190708
유사도 점수: 1.498626708984375
------------------------------------------------------------
[결과 4]
상품정보: 오난코리아의 None, 선인장 가습기 N9-H2O(화이트/그레이/핑크)
상품번호: 0000202984
유사도 점수: 1.5188307762145996
------------------------------------------------------------
[결과 5]
상품정보: 한일의 None, 에어미스트촉촉 간편세척 초음파 가습기
상품번호: 0000216148
유사도 점수: 1.5265119075775146
------------------------------------------------------------
